<a href="https://colab.research.google.com/github/Blusooyeon/bigdata-analysist-certificate/blob/main/practice_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 작업 1유형
데이터 설명 : 은행의 전화 마케팅에 대해 고객의 반응 여부            

dataurl : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')

In [2]:
df.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


In [5]:
#1. 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 인원을 가진 나이대는? (0~9 : 0 , 10~19 : 10)
result = (df.age//10 * 10).value_counts().index[0]
print(result)

30


In [7]:
#2. 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 나이대 구간의 인원은 몇명인가?
result = (df.age//10 * 10).value_counts().values[0]
print(result)

5056


In [11]:
#3. 나이가 25살 이상 29살 미만인 응답 고객들중 housing컬럼의 값이 yes인 고객의 수는?
result = df[(df.age >= 25) & (df.age < 29) & (df.housing == 'yes')].shape[0]
print(result)

504


In [12]:
#4. numeric한 값을 가지지 않은 컬럼들중 unique한 값을 가장 많이 가지는 컬럼은?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12870 entries, 0 to 12869
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         12870 non-null  int64 
 1   age        12870 non-null  int64 
 2   job        12870 non-null  object
 3   marital    12870 non-null  object
 4   education  12870 non-null  object
 5   default    12870 non-null  object
 6   balance    12870 non-null  int64 
 7   housing    12870 non-null  object
 8   loan       12870 non-null  object
 9   contact    12870 non-null  object
 10  day        12870 non-null  int64 
 11  month      12870 non-null  object
 12  campaign   12870 non-null  int64 
 13  pdays      12870 non-null  int64 
 14  previous   12870 non-null  int64 
 15  poutcome   12870 non-null  object
 16  y          12870 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.7+ MB


In [13]:
df.nunique()

ID           12870
age             76
job             12
marital          3
education        4
default          2
balance       4188
housing          2
loan             2
contact          3
day             31
month           12
campaign        36
pdays          461
previous        30
poutcome         4
y                2
dtype: int64

In [14]:
lst= []
for col in df.select_dtypes(exclude='int'):
    target = df[col]
    lst.append([col,target.nunique()])

dfs  = pd.DataFrame(lst).sort_values(1,ascending=False)
result = dfs[dfs[1] ==dfs[1].max()][0].values
print(result)

['job' 'month']


In [15]:
#5. balance 컬럼값들의 평균값 이상을 가지는 데이터를 ID값을 기준으로 내림차순 정렬했을때 상위 100개 데이터의 balance값의 평균은?
result = df[df.balance >= df.balance.mean()].sort_values('ID',ascending=False).head(100).balance.mean()
print(result)

3473.73


In [16]:
#6. 가장 많은 광고를 집행했던 날짜는 언제인가? (데이터 그대로 일(숫자),달(영문)으로 표기)
result = df[['day','month']].value_counts().index[0]
print(result)

(15, 'may')


In [18]:
#7. 데이터의 job이 unknown 상태인 고객들의 age 컬럼 값의 정규성을 검정하고자 한다. 샤피로 검정의 p-value값을 구하여라
from scipy.stats import shapiro
shapiro(df[df.job == 'unknown']['age'])[1]

0.19612720608711243

In [20]:
#8. age와 balance의 상관계수를 구하여라
df[['age', 'balance']].corr().iloc[0,1]

0.10198734763981462

In [23]:
#9. y 변수와 education 변수는 독립인지 카이제곱검정을 통해 확인하려한다. p-value값을 출력하라
from scipy.stats import chi2_contingency
chi2 , p ,dof, expected = chi2_contingency(pd.crosstab(df.y , df.education))
print(p)

7.901201277473551e-29


In [24]:
#10. 각 job에 따라 divorced/married 인원의 비율을 확인 했을 때 그 값이 가장 높은 값은?
t = df.groupby(['job','marital']).size().reset_index()
pivot = t.pivot_table(index='job', columns='marital')[0]
pivot = pivot.fillna(0)
pivot['ratio'] = pivot['divorced'] / pivot['married']

pivot.sort_values('ratio').ratio.values[-1]

0.2831050228310502


#### 작업 2유형
데이터 출처 : https://archive.ics.uci.edu/ml/datasets/Bank+Marketing (후처리 작업)            

데이터 설명 : 은행의 전화 마케팅에 대해 고객의 반응 여부


In [25]:
import pandas as pd
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
submission= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv')

display(train.head())
display(test.head())
display(submission.head())

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome
0,53608,32,management,single,tertiary,no,12569,no,no,cellular,1,jul,2,295,2,success
1,51055,25,services,single,secondary,no,801,no,no,cellular,5,jun,2,-1,0,unknown
2,52573,46,blue-collar,married,secondary,no,1728,yes,no,unknown,26,may,2,-1,0,unknown
3,50458,39,management,divorced,secondary,no,51,no,no,unknown,17,jun,2,-1,0,unknown
4,52272,31,services,single,tertiary,no,1626,no,no,unknown,31,jul,1,-1,0,unknown


,ID,predict
0,53608,0.0
1,51055,0.0
2,52573,0.0
3,50458,0.0
4,52272,0.0


In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12870 entries, 0 to 12869
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         12870 non-null  int64 
 1   age        12870 non-null  int64 
 2   job        12870 non-null  object
 3   marital    12870 non-null  object
 4   education  12870 non-null  object
 5   default    12870 non-null  object
 6   balance    12870 non-null  int64 
 7   housing    12870 non-null  object
 8   loan       12870 non-null  object
 9   contact    12870 non-null  object
 10  day        12870 non-null  int64 
 11  month      12870 non-null  object
 12  campaign   12870 non-null  int64 
 13  pdays      12870 non-null  int64 
 14  previous   12870 non-null  int64 
 15  poutcome   12870 non-null  object
 16  y          12870 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.7+ MB


In [28]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4291 entries, 0 to 4290
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         4291 non-null   int64 
 1   age        4291 non-null   int64 
 2   job        4291 non-null   object
 3   marital    4291 non-null   object
 4   education  4291 non-null   object
 5   default    4291 non-null   object
 6   balance    4291 non-null   int64 
 7   housing    4291 non-null   object
 8   loan       4291 non-null   object
 9   contact    4291 non-null   object
 10  day        4291 non-null   int64 
 11  month      4291 non-null   object
 12  campaign   4291 non-null   int64 
 13  pdays      4291 non-null   int64 
 14  previous   4291 non-null   int64 
 15  poutcome   4291 non-null   object
dtypes: int64(7), object(9)
memory usage: 536.5+ KB


In [29]:
from sklearn.model_selection import train_test_split

x = train.drop(columns = ['ID','y'])
xd = pd.get_dummies(x)
y = train['y']

x_train,x_test,y_train,y_test = train_test_split(xd,y,stratify =y ,random_state=1)


In [30]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
pred = rf.predict_proba(x_test)


In [31]:
from sklearn.metrics import roc_auc_score, classification_report

print(roc_auc_score(y_test,pred[:,1]))

0.7744149965220416


In [33]:
test_pred = rf.predict_proba(pd.get_dummies(test.drop(columns=['ID'])))
submission['predict'] = test_pred[:,1]

print('submission file')
display(submission.head())
submission.to_csv('00000000000000.csv',index=False)

submission file


,ID,predict
0,53608,0.73
1,51055,0.77
2,52573,0.02
3,50458,0.21
4,52272,0.32
